In [7]:

Orcestration_agent_prompt = '''
            "system",
            "You are a helpful assistant tasked with answering user questions. "
            "You have access to two tools: RAG , web_search_agent and get_current_weather "
            "For any user questions about LLM agents, use the RAG tool to get information for a vectorstore. "
            "For any questions, such as questions about current events, use the web_search tool to get information from the web. ",
            "For any question asked about the weather you use get_current_weather tool to get the answer"
        
    '''

web_search_agent_prompt = '''
You are an assistant for summarization task for a simple docs
you have to summarize the given context into a paragraph of 50 - 250 words.
you don't have to focus on the grammar just the prority is to capture the context in the summary
<context>
{context}
</context>
'''

In [8]:
from rag import RAG

In [9]:
RAG('How does machine learning algoritm works?')

'Machine learning algorithms process numbers, so text is converted to numbers by creating a vocabulary and assigning integer indices. The algorithms essentially minimize an objective function to achieve the desired outcome. They also use reward shaping to give incremental rewards for progress in learning.'

In [10]:
import shutil
shutil.rmtree('__pycache__', ignore_errors=True)

from prompt import web_search_agent_prompt

In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool
from typing import Annotated, List, Tuple, Union

@tool
def get_current_weather(city : Annotated[str, "city to get the weather for the city"]) -> str:
    """
    Get the current weather for a city.

    Parameters:
    city (str): The name of the city.

    Returns:
    str: The current temperature in the specified city.
    """
    return f"The current temperature in {city} is Sunny"


url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
from groq import Groq

@tool
def web_search_agent(url : Annotated[str, "url to retrive informatinfor a webpage"]):
    # Load documents from the URLs
    """
    get the information from the web page from given url

    Parameters:
    url (str): The name of the city.

    Returns:
    str: the text retrived form the url
    """
    client = Groq(
    api_key="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"
    )
    docs = WebBaseLoader(url).load()
    if docs:
        docs_list = [item for item in docs ]

        if len(docs_list) > 30:
            docs_list = docs_list[1:20]
    
    chat_prompt = web_search_agent_prompt.format(context=docs_list)

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role" : "system",
                "content": chat_prompt
            }
        ],
        model="llama-3.3-70b-versatile"
    )
    return chat_completion.choices[0].message.content




USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
from langchain_core.tools import Tool
weather_tool = Tool(
    name="Weather",
    func=get_current_weather,
    description="Provides current weather information for a location."
)
rag_tool = Tool(
    name="RAG",
    func=RAG,
    description="Retrieves documents based on a query."
)

web_search_tool = Tool(
    name="web_serach",
    func=web_search_agent,
    description="Retrieves documents from the give url."
)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent,AgentExecutor
prompt = ChatPromptTemplate.from_messages([("system",Orcestration_agent_prompt),
                                           ("human","{input}"),
                                           ("placeholder","{agent_scratchpad}")])

tools = [rag_tool,weather_tool,web_search_tool]

llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768",api_key="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")

search_agent = create_tool_calling_agent(llm,tools,prompt)

In [14]:
search_agent_exe = AgentExecutor(agent=search_agent,tools=tools)
search_agent_exe.invoke({"input":"What is an intelligent agent in artificial intelligence?"})

{'input': 'What is an intelligent agent in artificial intelligence?',
 'output': 'In artificial intelligence, an intelligent agent is a software agent that perceives its environment and takes actions autonomously to achieve goals. It can improve its performance over time and is designed to function without human intervention. An example of an intelligent agent is a chatbot that learns from user interactions to provide more accurate and relevant responses.'}

In [15]:
search_agent_exe.invoke({"input":"get the info from the following url 'https://medium.com/the-ai-forum/langchain-tooling-using-groq-d60ce5117710'"})


{'input': "get the info from the following url 'https://medium.com/the-ai-forum/langchain-tooling-using-groq-d60ce5117710'",
 'output': 'LLMs, or Large Language Models, can be transformed into autonomous agents using the ReAct pattern, which allows the model to "think out loud" and interact with dynamic environments. LLMs can also be part of larger agent models with capabilities such as natural language interfaces and goal-oriented actions. This information was provided by the RAG tool.'}

In [16]:
search_agent_exe.invoke({"input":"what is the weather in dublin"})

{'input': 'what is the weather in dublin',
 'output': 'The current temperature in Dublin is Sunny.'}

In [17]:
search_agent_exe.invoke({"input":"what is the weather in new york"})


{'input': 'what is the weather in new york',
 'output': 'The current temperature in New York is Sunny.'}